In [8]:
from pathlib import Path
import prody
import numpy as np
import torch
import dgl
import pickle

from typing import Iterable, Union, List, Dict

class Unparsable(Exception):
    pass

prody.confProDy(verbosity='none')

#processing code adapted from Ian Dunn: https://github.com/Dunni3/keypoint-diffusion/blob/main/data_processing/pdbbind_processing.py

#hard-code in element map for now (find all elements across pockets and hard code it)

In [2]:
#function to parse the receptors
def parse_pocket(pocket_path): #reads in pdb file of a receptor(binding pocket) into a prody AtomGroup
    
    receptor = prody.parsePDB(str(pocket_path))

    if receptor is None: #errors in reading in a pocket
        raise Unparsable
    
    return receptor

#function to return atom positions, features
def get_pocket_atoms(rec_atoms, element_map):

    #position, features and indices for all pocket atoms
    rec_atom_positions = rec_atoms.getCoords()
    rec_atom_features, other_atoms_mask = receptor_featurizer(element_map=element_map, rec_atoms=rec_atoms)

    #convert positions and features to tensors
    rec_atom_positions = torch.tensor(rec_atom_positions).float()
    rec_atom_features = torch.tensor(rec_atom_features).float()

    # remove "other" atoms from the receptor
    rec_atom_positions = rec_atom_positions[~other_atoms_mask]
    rec_atom_features = rec_atom_features[~other_atoms_mask]

    return rec_atom_positions, rec_atom_features


#function to featurize the receptor atoms
def receptor_featurizer(element_map, rec_atoms, protein_atom_elements = None):

    if rec_atoms is None and protein_atom_elements is None:
        raise ValueError
    
    if protein_atom_elements is None:
        protein_atom_elements: np.ndarray = rec_atoms.getElements()

    #one-hot encode atom elements
    onehot_elements = one_hot_encode(protein_atom_elements, element_map)

    #mask "other" atoms
    other_atoms_mask = torch.tensor(onehot_elements[:, -1] == 1).bool()

    #remove "other" category from onehot_elements, assuming they are last in the one-hot encoding
    protein_atom_features = onehot_elements[:, :-1]

    return protein_atom_features, other_atoms_mask


#function to one-hot encode all atoms of the receptor
def one_hot_encode(atom_elements: Iterable, element_map: Dict[str, int]):

    def element_to_idx(element_str, element_map=element_map):
        try:
            return element_map[element_str]
        except KeyError:
            return element_map['other']

    element_idxs = np.fromiter((element_to_idx(element) for element in atom_elements), int)
    onehot_elements = np.zeros((element_idxs.size, len(element_map.values())))
    onehot_elements[np.arange(element_idxs.size), element_idxs] = 1

    return onehot_elements

#function to build a graph from receptor atoms using dgl
def build_pocket_graph(atom_positions: torch.Tensor, atom_features: torch.Tensor, k: int, edge_algorithm: str):
    #add functionality for radius graphs too

    g = dgl.knn_graph(atom_positions, k=k, algorithm=edge_algorithm, dist='euclidean', exclude_self=True)
    g.ndata['x_0'] = atom_positions
    g.ndata['h_0'] = atom_features
    
    return g

In [3]:
#graph class to read in pocket and compute pocket representation

class GraphPocket:
#callable class to read a pocket and output the graph
    
    def __init__(self):

        #hard code element map and k for graph
        self.rec_elements = {'C': 0, 'N': 1, 'O': 2, 'S': 3}
        self.threshold_k = 3
        self.algorithm = 'bruteforce-blas'

    def __call__(self, pocket_path):

        pocket = parse_pocket(pocket_path)
        positions, features = get_pocket_atoms(pocket, self.rec_elements)

        graph = build_pocket_graph(positions, features, self.threshold_k, self.algorithm)

        return graph
        

In [4]:
pocket_to_graph = GraphPocket()

graph = pocket_to_graph(pocket_path='../../TOUGH-M1/data/11asA/11asA_pocket.pdb')

print(graph)

Graph(num_nodes=357, num_edges=1071,
      ndata_schemes={'x_0': Scheme(shape=(3,), dtype=torch.float32), 'h_0': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})


In [5]:
#build a class for a tuple dataset from the positive and negative pairs

class GraphTupleDataset(dgl.data.DGLDataset):

    def __init__(self, name, pocket_list, pos_list, neg_list):
        self.pocket_list = pocket_list
        self.pos_list = pos_list
        self.neg_list = neg_list
        self.graphs = []
        self.labels = []

        #filter pos and neg based on pocket_list

        self.pocket_index_map = {}

        super().__init__(name=name)

    def __getitem__(self, index):

        pair, label = self.labels[index]
        pocket1 = self.graphs[self.pocket_index_map[pair[0]]]
        pocket2 = self.graphs[self.pocket_index_map[pair[1]]]

        return (pocket1, pocket2), label

    def __len__(self):

        return len(self.labels)
    
    def process(self):

        pocket_to_graph = GraphPocket()

        for i, pocket in enumerate(self.pocket_list):
            graph = pocket_to_graph(pocket_path=f'../../TOUGH-M1/data/{pocket}/{pocket}_pocket.pdb')
            self.graphs.append(graph)
            self.pocket_index_map[pocket] = i

        for pos_pair in self.pos_list:
            self.labels.append((pos_pair, 1))
        for neg_pair in self.neg_list:
            self.labels.append((neg_pair, 0))



In [ ]:
##function to read pocket from graph, call the graphpocket class and generate a graph
from sklearn.model_selection import KFold, GroupShuffleSplit

#function for dataloading tuples of the pockets from pocket lists - used to get dataloader from a dataset class

def create_dataset(pos_path, neg_path, fold_nr, type, n_folds=5, seed=42):
    
    #load in the list of pocket and corresponding sequence clusters
    with open('../cluster_map.pkl', 'rb') as file:
        pocket_seq = pickle.load(file)

    pocket_list = [pdb[0] + pdb[1] for pdb in list(pocket_seq.keys())]

    pockets = list(pocket_seq.keys())
    clusters = list(pocket_seq.values())

    if type == 'seq':
        split = GroupShuffleSplit(n_splits=n_folds, test_size=1.0/n_folds, random_state=seed)
        folds = list(split.split(pockets, groups=clusters))
        train_index, test_index = folds[fold_nr] #fold number?
        pocket_train, pocket_test = [pocket_list[i] for i in train_index], [pocket_list[i] for i in test_index]
    
    if type == 'random':
        split = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
        folds = list(split.split(pocket_list))
        train_index, test_index = folds[fold_nr]
        pocket_train, pocket_test = [pocket_list[i] for i in train_index], [pocket_list[i] for i in test_index]


    with open(pos_path) as f:
        pos_pairs = [line.split()[:2] for line in f.readlines()]
    with open(neg_path) as f:
        neg_pairs = [line.split()[:2] for line in f.readlines()]

    train_dataset = GraphTupleDataset(name='train', pocket_list=pocket_train, pos_list=pos_pairs, neg_list=neg_pairs)

    test_dataset = GraphTupleDataset(name='test', pocket_list=pocket_test, pos_list=pos_pairs, neg_list=neg_pairs)

    return train_dataset, test_dataset

def get_dataloader():

    return

In [6]:
#test dataset

with open('../data/TOUGH-M1_positive.list') as f:
    pos_pairs = [line.split()[:2] for line in f.readlines()]
with open('../data/TOUGH-M1_positive.list') as f:
    neg_pairs = [line.split()[:2] for line in f.readlines()]
with open('../data/TOUGH-M1_pocket.list') as f:
    pocket_list = [line.split()[0] for line in f.readlines()]

print(pocket_list[:3])
print(pos_pairs[:3])

pocket_test = pocket_list[:3]
pos_test = [['11asA', '13pkD'],['11asA', '154lA']]
neg_test = [['13pkD', '154lA']]

dataset = GraphTupleDataset(name='mock_dataset', pocket_list=pocket_test, pos_list=pos_test, neg_list=neg_test)

sample, label = dataset[0]

print(f'Sample shape: {sample[0].ndata}, {sample[1].ndata}, Label: {label}')

# Print dataset length
print(f'Dataset length: {len(dataset)}')

['11asA', '13pkD', '154lA']
[['1lkxD', '2qenA'], ['1lkxD', '3zl8A'], ['1lkxD', '4dz6A']]
Sample shape: {'x_0': tensor([[34.0880, 20.7360,  4.2120],
        [32.8390, 19.9900,  4.1150],
        [32.0280, 20.4300,  2.8990],
        ...,
        [21.0980, 30.6810, 13.8420],
        [20.1760, 30.9780, 13.0940],
        [21.9320, 29.6830, 13.5870]]), 'h_0': tensor([[0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.]])}, {'x_0': tensor([[ 6.9000e-01,  9.0423e+01,  1.1705e+02],
        [-1.2700e-01,  8.9319e+01,  1.1751e+02],
        [-3.4000e-01,  8.9321e+01,  1.1900e+02],
        [-4.8600e-01,  9.0376e+01,  1.1962e+02],
        [-3.8600e-01,  8.8130e+01,  1.1958e+02],
        [-5.7600e-01,  8.7973e+01,  1.2101e+02],
        [ 5.4800e-01,  8.7081e+01,  1.2156e+02],
        [ 1.6740e+00,  8.7134e+01,  1.2106e+02],
        [-1.9290e+00,  8.7375e+01,  1.2127e+02],
        [ 1.6800e-01,  9.4821e+01,  1.1731e+02],

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

nx_g = graph.to_networkx().to_undirected()

# Step 2: Extract Node Features (here, using 'x_0' for coloring)
# Normalize features for coloring or choose a specific feature
node_colors = graph.ndata['x_0'][:, 0].numpy()  # Use the first feature for coloring
node_colors = (node_colors - node_colors.min()) / (node_colors.max() - node_colors.min())

# Step 3: Visualize the Graph
plt.figure(figsize=(8, 8))
# Position nodes using the Spring layout
pos = nx.spring_layout(nx_g, seed=42)
# Draw the nodes (you can map node_colors to any colormap you like)
for edge in nx_g.edges():
    points = np.array([pos[edge[0]], pos[edge[1]]])
    plt.plot(points[:, 0], points[:, 1], 'k-', lw=0.5)

# Draw nodes
for node in nx_g.nodes():
    plt.scatter(pos[node][0], pos[node][1], c='b', s=50)

plt.show()

In [ ]:
from Bio.PDB import PDBParser

parser = PDBParser()

structure = parser.get_structure(id = '1lasA', file = '../../TOUGH-M1/data/11asA/11asA_pocket.pdb')

atom_coords = []
for model in structure:
    for chain in model:
        for residue in chain:
            for atom in residue:
                atom_coords.append(atom.get_coord())  # Get 3D coordinates of each atom

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot atom positions
x, y, z = zip(*atom_coords)
ax.scatter(x, y, z)

ax.scatter(x, y, z, color='r')  # 'r' for nodes, change as needed

# Optionally, draw edges
# For demonstration, let's assume we have a simple edge list
edges = [(0, 1), (1, 2), (2, 3)]  # Example edges, replace with your graph's edges
for edge in edges:
    xs, ys, zs = zip(*[atom_coords[edge[0]], atom_coords[edge[1]]])
    ax.plot(xs, ys, zs, color='b')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

In [ ]:
src, dst = graph.edges()

# Convert the tensors to lists for easy handling
src_list = src.tolist()
dst_list = dst.tolist()

# Combine the source and destination lists into a list of tuples representing edges
edge_list = list(zip(src_list, dst_list))

node_coords = np.array(atom_coords)  # Example, replace with actual node positions

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plotting graph nodes (overlain on the protein structure)
ax.scatter(node_coords[:, 0], node_coords[:, 1], node_coords[:, 2], color='r', s=50)  # Red nodes

# Plotting edges
for edge in edge_list:
    start_pos, end_pos = node_coords[edge[0]], node_coords[edge[1]]
    ax.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], [start_pos[2], end_pos[2]], color='g')  # Green edges

# You might want to adjust plot limits and other aesthetics
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

In [ ]:
import torch
import dgl 
from dgl.dataloading import GraphDataLoader

import pickle

In [ ]:
#shift the sequence cluster code here (provided you have the information)